# 1

In [9]:
import pandas as pd

# 构造模拟数据,包含一些重复、缺失和异常情况
data = {
    '商品ID': [1001, 1002, 1001, 1003, 1004], # 有重复商品ID
    '商品名称': ['商品A', '商品B', '商品A', '商品C', '商品D'],
    '价格': [19.9, 29.9, 19.9, 39.9, None], # 有缺失价格
    '销量': [100, 200, 100, 300, 400],
    '评论数': [50, 80, 50, 'abc', 100] # 有异常评论数
}

# 将数据转换为DataFrame
df = pd.DataFrame(data)

# 存储数据为csv文件
df.to_csv('ecommerce_sales_data.csv', encoding='utf-8', index=False)

# 读取csv文件数据
df_read = pd.read_csv('ecommerce_sales_data.csv')

# 找出重复数据
duplicated_rows = df_read[df_read.duplicated()]
print("重复数据:")
print(duplicated_rows)

# 删除重复
df_on_duplicates = df_read.drop_duplicates()
print("\n删除重复后数据:")
print(df_on_duplicates)

重复数据:
   商品ID 商品名称    价格   销量 评论数
2  1001  商品A  19.9  100  50

删除重复后数据:
   商品ID 商品名称    价格   销量  评论数
0  1001  商品A  19.9  100   50
1  1002  商品B  29.9  200   80
3  1003  商品C  39.9  300  abc
4  1004  商品D   NaN  400  100


# 4. 分箱

In [21]:
import numpy as np
import math

# 1. 原始数据
salary = np.array([2200,2300,2400,2500,
                   2500,2800,3000,3200,
                   3500,3800,4000,4500,
                   4700,4800,4900,5000])

# 2. 等深分箱(假设要分成4箱，每箱4条数据)
#   salary.size = 16, int(salary.size/4) = 4
depth = salary.reshape(int(salary.size / 4), 4)
print("等深分箱:")
print(depth)

# ============ 均值平滑 ============
mean_depth = np.full((depth.shape[0], depth.shape[1]), 0, dtype=float)
for i in range(depth.shape[0]):
    row_mean = depth[i].mean()  # 计算当前行（一个分箱）的均值
    for j in range(depth.shape[1]):
        mean_depth[i][j] = row_mean

print("\n等深分箱 -- 均值平滑:")
print(mean_depth)

# ============ 中值平滑 ============
median_depth = np.full((depth.shape[0], depth.shape[1]), 0, dtype=float)
for i in range(depth.shape[0]):
    row_median = np.median(depth[i])  # 计算当前行的中位数
    for j in range(depth.shape[1]):
        median_depth[i][j] = row_median

print("\n等深分箱 -- 中值平滑:")
print(median_depth)

# ============ 边界值平滑 ============
# 首先确定每个分箱的左右边界
edge_depth = np.full((depth.shape[0], depth.shape[1]), 0, dtype=float)
edge_left = np.zeros(depth.shape[0], dtype=float)
edge_right = np.zeros(depth.shape[0], dtype=float)

for i in range(depth.shape[0]):
    edge_left[i] = depth[i][0]    # 该分箱的最左值
    edge_right[i] = depth[i][-1]  # 该分箱的最右值

# 根据距离最近原则进行边界平滑
# 通常也会把“第 1 个元素直接用左边界，第 4 个元素直接用右边界”，中间元素则比较两边距离。
for i in range(depth.shape[0]):
    for j in range(depth.shape[1]):
        if j == 0:
            # 第 1 个元素，用左边界
            edge_depth[i][j] = edge_left[i]
        elif j == depth.shape[1] - 1:
            # 最后 1 个元素，用右边界
            edge_depth[i][j] = edge_right[i]
        else:
            # 中间元素，看它更接近左边界还是右边界
            left_dist = (depth[i][j] - edge_left[i]) ** 2
            right_dist = (depth[i][j] - edge_right[i]) ** 2
            if left_dist <= right_dist:
                edge_depth[i][j] = edge_left[i]
            else:
                edge_depth[i][j] = edge_right[i]

print("\n等深分箱 -- 边界值平滑:")
print(edge_depth)


等深分箱:
[[2200 2300 2400 2500]
 [2500 2800 3000 3200]
 [3500 3800 4000 4500]
 [4700 4800 4900 5000]]

等深分箱 -- 均值平滑:
[[2350. 2350. 2350. 2350.]
 [2875. 2875. 2875. 2875.]
 [3950. 3950. 3950. 3950.]
 [4850. 4850. 4850. 4850.]]

等深分箱 -- 中值平滑:
[[2350. 2350. 2350. 2350.]
 [2900. 2900. 2900. 2900.]
 [3900. 3900. 3900. 3900.]
 [4850. 4850. 4850. 4850.]]

等深分箱 -- 边界值平滑:
[[2200. 2200. 2500. 2500.]
 [2500. 2500. 3200. 3200.]
 [3500. 3500. 3500. 4500.]
 [4700. 4700. 5000. 5000.]]


# 6. 属性 A 和属性 B 的协方差与相关系数

In [4]:
import pandas as pd
import numpy as np

# 导入 pandas
# 导入 NumPy
a = [47, 83, 81, 18, 72, 41, 50, 66, 47, 20, 96, 21, 16, 60, 37, 59, 22, 16, 32, 63]
# 属性A的值
b = [56, 96, 84, 21, 87, 67, 43, 64, 85, 67, 68, 64, 95, 58, 56, 75, 6, 11, 68, 63]
# 属性B的值
data = np.array([a, b]).T
# 数据转置
dfab = pd.DataFrame(data, columns=['A','B'])
# 转为 DataFrame 格式
print('属性A和B的协方差:', dfab.A.cov(dfab.B))
# 输出属性A和B的协方差
print('属性A和B的相关系数:', dfab.A.corr(dfab.B))
# 输出属性A和B的相关系数

属性A和B的协方差: 310.2157894736842
属性A和B的相关系数: 0.49924871046524394


# 7

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_iris

# 导入MinMaxScaler包
# 导入鸢尾花数据集
iris=load_iris()
# 提取前6个数据
data=iris.data[0:6]

print(data)
# 输出未归一化的结果
mms=MinMaxScaler().fit_transform(data)

print(mms)
# 输出归一化的结果

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]]
[[0.625      0.55555556 0.25       0.        ]
 [0.375      0.         0.25       0.        ]
 [0.125      0.22222222 0.         0.        ]
 [0.         0.11111111 0.5        0.        ]
 [0.5        0.66666667 0.25       0.        ]
 [1.         1.         1.         1.        ]]


# 8. Z-score 规范化

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

iris = load_iris()
data = iris.data[0:6]
print(data)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]]


# 9. 方差筛选法

In [6]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import load_iris

iris = load_iris()

print("\n原始数据:\n",iris.data[0:5]) #输出原始数据

x_var=VarianceThreshold(threshold=0.5).fit_transform(iris.data)
print("方差筛选法过滤后数据\n",format(x_var[0:5])) #输出方差筛选法过滤后的数据


原始数据:
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
方差筛选法过滤后数据
 [[5.1 1.4 0.2]
 [4.9 1.4 0.2]
 [4.7 1.3 0.2]
 [4.6 1.5 0.2]
 [5.  1.4 0.2]]


# 10. 数据替换

In [7]:
import re

address = "江苏省徐州市鼓楼区幸福路碧园小区19栋18号1单元202室" #定义数据

address=re.sub('[0-9]','*',address)   #对地址中的数据进行替换

pattern = "区(.*?)路(.*?)小区"   #设置正则表达式

pat=re.findall(pattern,address)[0][::-1]   #查找反序中的字符

for i in pat:  #遍历字符
    if i in address:  #判断字符在address中
        address=address.replace(i,'*') #字符替换

print(address)  #输出结果

江苏省徐州市鼓楼区*路*小区**栋**号*单元***室


# 12

In [ ]:
import numpy as np

# 创建一个二维数组
a = np.array([[1, 2], [3, 4]])
print('数组a: {}'.format(a))

# 输出数组a的类型
print('数组a的类型为:{}'.format(type(a)))

# 输出数组a元素的类型
print('数组a元素的类型: {}'.format(type(a[0][0])))

# 输出数组a的维度
print('数组a的维度为: {}'.format(a.ndim))

# 设置数组中元素的类型
b = np.array([1, 2, 3, 4, 5], dtype=np.float64)
print('数组b: {}'.format(b))

# 输出数组b的类型
print('数组b的类型为: {}'.format(type(b)))

# 输出数组b的维度
print('数组b的维度为: {}'.format(b.ndim))

# 创建一个三维数组
c = np.array([[1, 2, 3, 4, 5]])
print('数组c: {}'.format(c))


数组a: [[1 2]
 [3 4]]
数组a的类型为:<class 'numpy.ndarray'>
数组a元素的类型: <class 'numpy.int64'>
数组a的维度为: 2
数组b: [1. 2. 3. 4. 5.]
数组b的类型为: <class 'numpy.ndarray'>
数组b的维度为: 1
数组c: [[1 2 3 4 5]]


# 13

In [ ]:
import numpy as np

a = np.zeros(5)
print('数组a为: {}'.format(a))

b = np.ones(5)
print('数组b为: {}'.format(b))

c = np.zeros((2, 2))
print('数组c为: {}'.format(c))

d = np.ones((2, 2))
print('数组d为: {}'.format(d))

e = np.eye(3)
print('数组e为: {}'.format(e))

f = np.arange(0, 10, 2)
print('数组f为: {}'.format(f))

g = np.full((4, 4), 6)
print('数组g为: {}'.format(g))

h = np.empty((3, 2))
print('数组h为: {}'.format(h))


数组a为: [0. 0. 0. 0. 0.]
数组b为: [1. 1. 1. 1. 1.]
数组c为: [[0. 0.]
 [0. 0.]]
数组d为: [[1. 1.]
 [1. 1.]]
数组e为: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
数组f为: [0 2 4 6 8]
数组g为: [[6 6 6 6]
 [6 6 6 6]
 [6 6 6 6]
 [6 6 6 6]]
数组h为: [[7.35460957e-316 0.00000000e+000]
 [2.14321575e-312 1.63041663e-322]
 [7.36550772e-316 6.81607814e-310]]


# 14

In [20]:
import numpy as np
a = np.array([2,1,1,2])
b = np.array([2,1,1,2])
print('两个相同形状的数组a,b相加:')
c = a + b
print('{}+{}={}'.format(a,b,c))
print('两个相同形状的数组a,b相减:')
c = a - b
print('{}-{}={}'.format(a,b,c))
print('两个相同形状的数组a,b相乘:')
c = a * b
print('{}*{}={}'.format(a,b,c))
print('两个相同形状的数组a,b相除:')
c = a / b
print('{}/{}={}'.format(a,b,c))
print('数组a的相反数:')
print(-a)
print('数组a的平方:')
print(a**2)
print('数组a的按位异或:')
print(a^b)


两个相同形状的数组a,b相加:
[2 1 1 2]+[2 1 1 2]=[4 2 2 4]
两个相同形状的数组a,b相减:
[2 1 1 2]-[2 1 1 2]=[0 0 0 0]
两个相同形状的数组a,b相乘:
[2 1 1 2]*[2 1 1 2]=[4 1 1 4]
两个相同形状的数组a,b相除:
[2 1 1 2]/[2 1 1 2]=[1. 1. 1. 1.]
数组a的相反数:
[-2 -1 -1 -2]
数组a的平方:
[4 1 1 4]
数组a的按位异或:
[0 0 0 0]


# 15

In [ ]:
import numpy as np

a = np.array([2, 1, 1, 2])
b = np.array([2, 1, 1, 2])
print('两个相同形状的数组a,b相加:')
c = a + b
print('{} + {} = {}'.format(a, b, c))
print('两个相同形状的数组a,b相减:')
c = a - b
print('{} - {} = {}'.format(a, b, c))
print('两个相同形状的数组a,b相乘:')
c = a * b
print('{} * {} = {}'.format(a, b, c))
print('两个相同形状的数组a,b相除:')
c = a / b
print('{} / {} = {}'.format(a, b, c))
print('数组a的相反数:')
print(-a)
print('数组a的平方:')
print(a**2)
print('数组a的按位异或:')
print(a^b)


两个相同形状的数组a,b相加:
[2 1 1 2] + [2 1 1 2] = [4 2 2 4]
两个相同形状的数组a,b相减:
[2 1 1 2] - [2 1 1 2] = [0 0 0 0]
两个相同形状的数组a,b相乘:
[2 1 1 2] * [2 1 1 2] = [4 1 1 4]
两个相同形状的数组a,b相除:
[2 1 1 2] / [2 1 1 2] = [1. 1. 1. 1.]
数组a的相反数:
[-2 -1 -1 -2]
数组a的平方:
[4 1 1 4]
数组a的按位异或:
[0 0 0 0]


# 16

In [2]:
import numpy as np

a = np.array([[1, 1, 2, 2], [3, 3, 4, 4]])
print('数组a : {}'.format(a))
print('取数组a[1,3]的数据 : {}'.format(a[1,3]))

a[1,3] = 666
print('更改后的数组a : {}'.format(a))

b = a[:, :2]
print('数组b : {}'.format(b))


数组a : [[1 1 2 2]
 [3 3 4 4]]
取数组a[1,3]的数据 : 4
更改后的数组a : [[  1   1   2   2]
 [  3   3   4 666]]
数组b : [[1 1]
 [3 3]]


# 17

In [19]:
import numpy as np

array_2d = np.arange(1, 10).reshape((3, 3))
print(array_2d)
# 访问索引为[0, 2]的元素
print(array_2d[[0, 2]])

# 使用两个花式索引访问元素
print(array_2d[[0, 2], [1, 1]])


[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 2 3]
 [7 8 9]]
[2 8]


# 18

In [18]:
import numpy as np
arr_2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(arr_2d)
print(arr_2d[:2])
print(arr_2d[2, 0:1])
print(arr_2d[2, 1:])


[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 2 3]
 [4 5 6]]
[7]
[8 9]


# 19

In [17]:
import numpy as np
import pandas as pd
s = pd.Series([1, 2, 3])
print(s)
print('使用一维数组创建Series')
arr = np.array([1, 2, 3])  # 一维数组
ys = pd.Series(arr)
print(ys)
print('使用字典创建Series')
dic = {"index1": 1, "index2": 2, "index3": 3}
ds = pd.Series(dic)
print(ds)
print('使用标量创建Series')
bs = pd.Series(10)
print(bs)


0    1
1    2
2    3
dtype: int64
使用一维数组创建Series
0    1
1    2
2    3
dtype: int64
使用字典创建Series
index1    1
index2    2
index3    3
dtype: int64
使用标量创建Series
0    10
dtype: int64


# 20

In [15]:
import pandas as pd
import numpy as np
print("由Series组成的字典创建DataFrame")
data2 = {'a': pd.Series(np.random.rand(3)),
         'b': pd.Series(np.random.rand(4)),
         'c': pd.Series(np.random.rand(5))}
df2 = pd.DataFrame(data2)
print(df2)


由Series组成的字典创建DataFrame
          a         b         c
0  0.289125  0.595944  0.988483
1  0.958288  0.440776  0.772403
2  0.707241  0.389404  0.370199
3       NaN  0.505913  0.641038
4       NaN       NaN  0.300318


# 21. 数据分组

In [12]:
import pandas as pd

# 创建DataFrame
df = pd.DataFrame({
    '班级': ['A', 'B', 'A', 'B'],
    '姓名': ['小红', '小明', '小李', '小夏'],
    '语文': [90, 80, 64, 71],
    '数学': [94, 81, 62, 51],
    '英语': [98, 82, 64, 61],
    '评价': ['优秀', '优秀', '合格', '合格']
})

# 按评价列分组
grouped = df.groupby('评价')

# 输出结果
print("输出df")
print(df)
print("\n以下为分组后的输出")
print(grouped.get_group('优秀'))


输出df
  班级  姓名  语文  数学  英语  评价
0  A  小红  90  94  98  优秀
1  B  小明  80  81  82  优秀
2  A  小李  64  62  64  合格
3  B  小夏  71  51  61  合格

以下为分组后的输出
  班级  姓名  语文  数学  英语  评价
0  A  小红  90  94  98  优秀
1  B  小明  80  81  82  优秀
